In [0]:
#Importing the Keras Libraries and packages
from keras.models import Sequential #initializes the NN in a sequential manner
from keras.layers import Convolution2D #for the convolutional layers and 2D images--deals with images
from keras.layers import MaxPooling2D #for the Pooling layers
from keras.layers import Flatten #for the flattening layers--converts the pooling features into large connected vectors that act as inputs to the NN
from keras.layers import Dense #adds the fully connected layers into an ANN


Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving cat&dog.zip to cat&dog.zip


In [0]:
!unzip -q "/content/cat&dog.zip"

In [0]:
import os
os.path.isfile("drive/Colab Notebooks/content/cat&dog")

False

In [0]:
#Initialising the CNN
classifier = Sequential() #an object of the class

In [0]:
#Step-1 :- Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu')) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  


In [0]:
#Step-2 Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [0]:
#Step-3 Flattening
classifier.add(Flatten())

In [0]:
#Step-4 Full Connection
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=1,activation='sigmoid'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#Step-5 Compiling the CNN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
#Fitting images to CNN
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
training_set = train_datagen.flow_from_directory(
        '/content/cat&dog/training',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [0]:
test_set= test_datagen.flow_from_directory(
        '/content/cat&dog/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [0]:
classifier.fit_generator(
        training_set,
        samples_per_epoch=8000,
        nb_epoch=25,
        validation_data=test_set,
        nb_val_samples=2000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=250, epochs=25, validation_steps=2000)`
  


Epoch 1/25
250/250 [==============================] - 280s 1s/step - loss: 0.6719 - accuracy: 0.5936 - val_loss: 0.6575 - val_accuracy: 0.5681
Epoch 2/25
250/250 [==============================] - 283s 1s/step - loss: 0.5487 - accuracy: 0.7244 - val_loss: 0.4145 - val_accuracy: 0.6500
Epoch 3/25
250/250 [==============================] - 280s 1s/step - loss: 0.4698 - accuracy: 0.7802 - val_loss: 0.5925 - val_accuracy: 0.6830
Epoch 4/25
250/250 [==============================] - 279s 1s/step - loss: 0.4237 - accuracy: 0.8031 - val_loss: 0.6503 - val_accuracy: 0.6830
Epoch 5/25
250/250 [==============================] - 285s 1s/step - loss: 0.3750 - accuracy: 0.8343 - val_loss: 0.5585 - val_accuracy: 0.6882
Epoch 6/25
250/250 [==============================] - 280s 1s/step - loss: 0.3426 - accuracy: 0.8511 - val_loss: 1.0061 - val_accuracy: 0.6738
Epoch 7/25
250/250 [==============================] - 286s 1s/step - loss: 0.3122 - accuracy: 0.8660 - val_loss: 0.4037 - val_accuracy: 0.7130

In [0]:
classifier.save("model.h5")
print("Saved")

Saved


In [0]:
from flask import Flask, request, jsonify, render_template
import base64, json
from io import BytesIO
import numpy as np

In [0]:
# declare constants
HOST = '0.0.0.0'
PORT = 8888

In [0]:
# initialize flask application
app = Flask('catdogclassifier')

In [0]:
# Read model to keep it ready all the time
model =('/content/model.h5')
CLASS_MAPPING = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghnqrt'

In [0]:
# Application template html/css/js
@app.route('/')
def home():
    return render_template("home.html")

In [0]:

 # home page
@app.route("/path1", endpoint='func1')
def home():
   return render_template('dogcat.html')

In [0]:
@app.route('/predict', methods=['GET','POST'])
def predict():
    results = {"prediction" :"Empty", "probability" :{}}

    # get data
    input_img = BytesIO(base64.urlsafe_b64decode(request.form['img']))

    # model.predict method takes the raw data and output a vector of probabilities
    res =  model.predict(input_img)

    results["prediction"] = str(CLASS_MAPPING[np.argmax(res)])
    results["probability"] = float(np.max(res))*100
    # results["prediction"] = 5
    # results["probability"] = 50.424

    # output data
    return json.dumps(results)

In [0]:
if __name__ == '__main__':
    # run web server
    app.run(host=HOST,
            debug=True,  # automatic reloading enabled
            port=PORT)

 * Serving Flask app "catdogclassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:8888/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
